<a href="https://colab.research.google.com/github/r-u-5k/study-dev/blob/master/bigdata/5-2-type2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 에어비엔비 가격?
- 평가: R-Squared, MAE, MSE, RMSE, RMSLE, MAPE
- target : price(가격)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,price
34323697,238
29927138,183
120362,234
~~~



## 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/ab_nyc/train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/ab_nyc/test.csv")
# train = pd.read_csv("train.csv")
# test =  pd.read_csv("test.csv")


##EDA

In [ ]:
train.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
0,35742633,Luxury new 1 bed apartment in East Williamsburg,265866685,Mindy,Brooklyn,Bushwick,40.69796,-73.92915,Entire home/apt,30,0,NaN,NaN,2,179,198
1,15840089,Kid- (and Adult-) Friendly Uptown 2-Bedroom,99602138,Yolanda,Manhattan,Washington Heights,40.83490,-73.94829,Entire home/apt,3,36,2019-01-01,1.13,1,5,150
2,24495605,Brooklyn Art Residence,75173477,Lara,Brooklyn,Bushwick,40.69932,-73.91191,Entire home/apt,3,7,2019-05-05,0.48,1,6,200
3,16238779,Spacious Studio Duplex in Heart of Williamsburg!,36382944,Joshua,Brooklyn,Williamsburg,40.71520,-73.96230,Entire home/apt,31,2,2018-01-01,0.06,1,0,230
4,25265496,Beautiful large 2 br appartement in Brooklyn!,22935245,Anne,Brooklyn,Bedford-Stuyvesant,40.69168,-73.95397,Entire home/apt,21,1,2018-07-14,0.08,1,25,145


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39116 entries, 0 to 39115
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              39116 non-null  int64  
 1   name                            39102 non-null  object 
 2   host_id                         39116 non-null  int64  
 3   host_name                       39100 non-null  object 
 4   neighbourhood_group             39116 non-null  object 
 5   neighbourhood                   39116 non-null  object 
 6   latitude                        39116 non-null  float64
 7   longitude                       39116 non-null  float64
 8   room_type                       39116 non-null  object 
 9   minimum_nights                  39116 non-null  int64  
 10  number_of_reviews               39116 non-null  int64  
 11  last_review                     31063 non-null  object 
 12  reviews_per_month               

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9779 entries, 0 to 9778
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              9779 non-null   int64  
 1   name                            9777 non-null   object 
 2   host_id                         9779 non-null   int64  
 3   host_name                       9774 non-null   object 
 4   neighbourhood_group             9779 non-null   object 
 5   neighbourhood                   9779 non-null   object 
 6   latitude                        9779 non-null   float64
 7   longitude                       9779 non-null   float64
 8   room_type                       9779 non-null   object 
 9   minimum_nights                  9779 non-null   int64  
 10  number_of_reviews               9779 non-null   int64  
 11  last_review                     7780 non-null   object 
 12  reviews_per_month               77

In [ ]:
train.isnull().sum()

,0
id,0
name,14
host_id,0
host_name,16
neighbourhood_group,0
neighbourhood,0
latitude,0
longitude,0
room_type,0
minimum_nights,0


In [ ]:
train['price'].describe()

,price
count,39116.000000
mean,153.057547
std,248.758522
min,0.000000
25%,69.000000
50%,105.000000
75%,175.000000
max,10000.000000


## 데이터 전처리 & 피처엔지니어링

In [ ]:
cols = train.select_dtypes(include='object').columns
cols

Index(['name', 'host_name', 'neighbourhood_group', 'neighbourhood',
       'room_type', 'last_review'],
      dtype='object')

In [ ]:
print(train.shape, test.shape)
train = train.drop(cols, axis=1)
train = train.drop('reviews_per_month', axis=1)
test = test.drop(cols, axis=1)
test = test.drop('reviews_per_month', axis=1)
print(train.shape, test.shape)

(39116, 16) (9779, 15)
(39116, 9) (9779, 8)


## 검증 데이터 분리

In [ ]:
target = train.pop('price')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=0)


In [ ]:
X_train.head()

,id,host_id,latitude,longitude,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
38456,6672812,1549512,40.68385,-73.96748,7,0,2,0
6768,13031826,18933388,40.67661,-73.95611,1,5,1,0
18400,16054732,4233535,40.68376,-73.95851,1,0,1,0
34747,18649105,44838931,40.76843,-73.93257,3,17,2,334
26528,15888227,11917115,40.71497,-73.96145,3,0,1,0


## 모델 & 평가

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=0)
model.fit(X_train, y_train)
pred = model.predict(X_val)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

r2 = r2_score(y_val, pred)
mae = mean_absolute_error(y_val, pred)
mse = mean_squared_error(y_val, pred)
rmse = root_mean_squared_error(y_val, pred)
print(r2)
print(mae)
print(mse)
print(rmse)

0.09808570975546438
78.41336400817997
61245.94182880878
247.47917453557335


## 예측 및 csv 제출

In [ ]:
pred = model.predict(test)

In [ ]:
pred[:10]

array([324.71,  64.78, 128.96, 118.2 , 107.52, 225.53,  88.85,  73.32,
        95.36,  88.1 ])

In [ ]:
submit = pd.DataFrame({'id': test['id'], 'output': pred})
submit.to_csv('result.csv', index=False)
pd.read_csv('result.csv')

,id,output
0,34323697,324.71
1,29927138,64.78
2,120362,128.96
3,31838301,118.20
4,7244824,107.52
...,...,...
9774,15359119,85.20
9775,23565653,219.11
9776,9920191,129.20
9777,4792342,120.66


##정리

In [ ]:
# y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/ab_nyc/y_test.csv")
# print(r2_score(y_test, pred))